<a href="https://colab.research.google.com/github/prantoran/ai-prac/blob/master/llm/synthetic_data/deep_eval_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


In [ ]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_text_splitters

In [ ]:
!pip3 install tiktoken -v

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/ToRead/RLSyllabus.pdf

drive/MyDrive/ToRead/RLSyllabus.pdf


In [ ]:
OPENAI_API_KEY = '...'

# Document Chunking

## Create chunks

In [ ]:
# Step 1. Chunk Documents
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter
import os

print(os.getcwd())

text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=0)

loader = PyPDFLoader("drive/MyDrive/ToRead/RLSyllabus.pdf")
raw_chunks = loader.load_and_split(text_splitter)


/content


## Convert chunks to embeddings

In [ ]:
!pip install langchain_openai

In [ ]:
from langchain_openai import OpenAIEmbeddings


embedding_model = OpenAIEmbeddings(api_key=OPEN_API_KEY)
content = [rc.page_content for rc in raw_chunks]
embeddings = embedding_model.embed_documents(content)


In [ ]:
len(embeddings)

2

In [ ]:
len(embeddings[0])

1536

In [ ]:
type(embeddings)

list

In [ ]:
type(embeddings[0])

list

# Context Generation

## Randomly select a chunk as focal anchor

In [ ]:
# Step 2: Generate context by selecting chunks
import random

reference_index = random.randint(0, len(embeddings) - 1)
reference_embedding = embeddings[reference_index]
contexts = [content[reference_index]]
print("Reference index: " + str(reference_index), "with contexts: ", contexts)


Reference index: 0 with contexts:  ['IEOR 8100: Advanced topics in Reinforcement Learning \nInstructor:  Shipra Agrawal \nMeeting time: Mondays and Wednesdays, 11:40 am - 12:55 pm, Location TBD \nIn this course, we will read recent research papers on reinforcement learning, with a particular emphasis \non advancements on reinforcement learning from human feedback (RLHF). RLHF has been popularized \nas a main method for aligning LLMs with human values and preferences. After a quick overview of basic \nconcepts in reinforcement learning, we will read some important papers on deep reinforcement \nlearning algorithms and analysis, followed by recent papers covering both empirical and theoretical \nfoundations of RLHF. These will be interspersed papers on relevant topics such as dueling bandits, \ninverse RL, imitation learning, multi-agent RL and offline RL, which form the technical basis of many \nrecent directions in RLHF. \nPrerequisites \nThis course is mainly aimed at PhD students or 

## Identify related chunks

In [ ]:
import numpy as np

similarity_threshold = 0.8
similar_indices = []
for i, embedding in enumerate(embeddings):
    product = np.dot(reference_embedding, embedding)
    norm = np.linalg.norm(reference_embedding) * np.linalg.norm(embedding)
    similarity = product / norm
    print(i, product, norm, similarity)
    if similarity >= similarity_threshold:
        similar_indices.append(i)

for i in similar_indices:
    contexts.append(content[i])
print(len(contexts))

0 0.9999998433066166 0.9999998433066165 1.0000000000000002
1 0.889226653766511 0.9999998753185456 0.8892267646365973
3


# Query Generation

Provide a prompt that asks the model to act as a copywriter, generating JSON objects containing an input key, which is the query. Each input should either be a question or statement answerable using the provided context.

Note: the focal anchor chunk is included twice.

In [ ]:
# Step 3. Generate a series of queries for similar chunks
from langchain_openai import ChatOpenAI
...

prompt = f"""I want you act as a copywriter. Based on the given context,
which is list of strings, please generate a list of JSON objects
with a `input` key. The `input` can either be a question or a
statement that can be addressed by the given context.

contexts:
{contexts}"""

print("prompt:\n", prompt)

query = ChatOpenAI(openai_api_key=OPEN_API_KEY).invoke(prompt)


prompt:
 I want you act as a copywriter. Based on the given context, 
which is list of strings, please generate a list of JSON objects 
with a `input` key. The `input` can either be a question or a 
statement that can be addressed by the given context.

contexts:
['IEOR 8100: Advanced topics in Reinforcement Learning \nInstructor:  Shipra Agrawal \nMeeting time: Mondays and Wednesdays, 11:40 am - 12:55 pm, Location TBD \nIn this course, we will read recent research papers on reinforcement learning, with a particular emphasis \non advancements on reinforcement learning from human feedback (RLHF). RLHF has been popularized \nas a main method for aligning LLMs with human values and preferences. After a quick overview of basic \nconcepts in reinforcement learning, we will read some important papers on deep reinforcement \nlearning algorithms and analysis, followed by recent papers covering both empirical and theoretical \nfoundations of RLHF. These will be interspersed papers on relevant t

In [ ]:
query

AIMessage(content='[\n    {\n        "input": "What are the prerequisites for IEOR 8100: Advanced topics in Reinforcement Learning?"\n    },\n    {\n        "input": "Can students suggest papers to read in the course on Reinforcement Learning?"\n    },\n    {\n        "input": "What is the course structure and requirements for IEOR 8100?"\n    },\n    {\n        "input": "What are some important papers on deep reinforcement learning algorithms and analysis?"\n    },\n    {\n        "input": "What is the focus of the course on reinforcement learning with human feedback (RLHF)?"\n    }\n]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 2197, 'total_tokens': 2322, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint

# Query Evolution

## Prompt Templates

Trying out multi-context understanding, multi-step reasoning, and hypothetical scenario. Other templates are also possible.

In [ ]:
# Evolution prompt templates as strings
multi_context_template = """
I want you to rewrite the given `input` so that it requires readers to use information from all elements in `Context`.

1. `Input` should require information from all `Context` elements.
2. `Rewritten Input` must be concise and fully answerable from `Context`.
3. Do not use phrases like 'based on the provided context.'
4. `Rewritten Input` should not exceed 15 words.

Context: {context}
Input: {original_input}
Rewritten Input:
"""

reasoning_template = """
I want you to rewrite the given `input` so that it explicitly requests multi-step reasoning.

1. `Rewritten Input` should require multiple logical connections or inferences.
2. `Rewritten Input` should be concise and understandable.
3. Do not use phrases like 'based on the provided context.'
4. `Rewritten Input` must be fully answerable from `Context`.
5. `Rewritten Input` should not exceed 15 words.

Context: {context}
Input: {original_input}
Rewritten Input:
"""

hypothetical_scenario_template = """
I want you to rewrite the given `input` to incorporate a hypothetical or speculative scenario.

1. `Rewritten Input` should encourage applying knowledge from `Context` to deduce outcomes.
2. `Rewritten Input` should be concise and understandable.
3. Do not use phrases like 'based on the provided context.'
4. `Rewritten Input` must be fully answerable from `Context`.
5. `Rewritten Input` should not exceed 15 words.

Context: {context}
Input: {original_input}
Rewritten Input:
"""


## Evolve Queries

In [ ]:
# Step 4. Evolve Queries
...

example_generated_query = "How do chatbots use natural language understanding?"
context = contexts
original_input = example_generated_query
evolution_templates = [multi_context_template, reasoning_template, hypothetical_scenario_template]

print("original_input:\n", original_input)
print("context:\n", context)
print("example_generated_query:\n", example_generated_query)
# Number of evolution steps to apply
num_evolution_steps = 3

# Function to perform random evolution steps
def evolve_query(original_input, context, steps):
    current_input = original_input
    for _ in range(steps):
        if type(current_input) is not str:
          current_input = current_input.content
        # Choose a random (or using custom logic) template from the list
        chosen_template = random.choice(evolution_templates)
        # Replace the placeholders with the current context and input
        evolved_prompt = chosen_template.replace("{context}", str(context))
        evolved_prompt = evolved_prompt.replace("{original_input}", current_input)
        print("evolved_prompt:\n", evolved_prompt)
        # Update the current input with the "Rewritten Input" section
        current_input = ChatOpenAI(openai_api_key=OPEN_API_KEY).invoke(evolved_prompt)
    return current_input

# Evolve the input by randomly selecting the evolution type
evolved_query = evolve_query(original_input, context, num_evolution_steps)


original_input:
 How do chatbots use natural language understanding?
context:
 ['IEOR 8100: Advanced topics in Reinforcement Learning \nInstructor:  Shipra Agrawal \nMeeting time: Mondays and Wednesdays, 11:40 am - 12:55 pm, Location TBD \nIn this course, we will read recent research papers on reinforcement learning, with a particular emphasis \non advancements on reinforcement learning from human feedback (RLHF). RLHF has been popularized \nas a main method for aligning LLMs with human values and preferences. After a quick overview of basic \nconcepts in reinforcement learning, we will read some important papers on deep reinforcement \nlearning algorithms and analysis, followed by recent papers covering both empirical and theoretical \nfoundations of RLHF. These will be interspersed papers on relevant topics such as dueling bandits, \ninverse RL, imitation learning, multi-agent RL and offline RL, which form the technical basis of many \nrecent directions in RLHF. \nPrerequisites \nThi

# Expected Ouput Generation

In [ ]:
# Step 5. Generate Expected Output
...

# Define prompt template
expected_output_template = f"""
I want you to generate an answer for the given `input`. This answer has to be factually aligned to the provided context.

Context: {context}
Input: {evolved_query}
Answer:
"""

# Fill in the values

prompt = expected_output_template.replace("{context}", str(context)).replace("{evolved_query}", evolved_query.content)

# Generate expected output
expected_output = ChatOpenAI(openai_api_key=OPEN_API_KEY).invoke(prompt)


In [ ]:
expected_output

AIMessage(content='Natural language understanding plays a crucial role in the progress and effectiveness of chatbot technology. By enabling chatbots to comprehend and respond to human language in a more nuanced and contextually appropriate manner, advancements in natural language understanding enhance the user experience by facilitating more engaging and productive interactions. These improvements can lead to more accurate responses, higher customer satisfaction, and increased efficiency in various applications such as customer service, information retrieval, and task automation. As chatbots continue to evolve, the integration of sophisticated natural language understanding capabilities will be key to further enhancing their performance and expanding their potential use cases.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 2409, 'total_tokens': 2526, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'aud

# Wrap Evolved Query, Context, and Expected Output

In [ ]:
from pydantic import BaseModel
from typing import Optional, List
...

class SyntheticData(BaseModel):
  query: str
  expected_output: Optional[str]
  context: List[str]

  def __init__(self, query, expected_output, context):
    super().__init__(
      query = query,
      expected_output = expected_output,
      context = context
    )

q = evolved_query
if type(q) is not str:
  q = q.content

out = expected_output
if type(out) is not str:
  out = out.content

synthetic_data = SyntheticData(
	query=q,
	expected_output=out,
	context=context
)

# Simple implementation of synthetic dataset
synthetic_dataset = []
synthetic_dataset.append(synthetic_data)


# DeepEval

In [ ]:
!pip install deepeval

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 21.7 MB/s eta 0:0

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer()
synthesizer.chunk_size = 200
synthesizer.chunk_overlap = 0
print(synthesizer.chunk_size)
synthesizer.generate_goldens_from_docs(
    document_paths=['drive/MyDrive/ToRead/quantum_cir.pdf'],
)


200
Event loop is already running. Applying nest_asyncio patch to allow async execution...


✨ 🧩 ✨ Generating Contexts: 100%|██████████| 9/9 [00:06<00:00,  1.34it/s]


Utilizing 8 out of 75 chunks.

✨ Generating up to 6 goldens using DeepEval (using gpt-4o and text-embedding-3-small, method=docs): 100%|██████████| 6/6 [00:36<00:00,  6.11s/it]


[Golden(input='Compare the feasibility and challenges of MPS vs. TN simulations in high-fidelity quantum circuits.', actual_output=None, expected_output='Matrix-product-state (MPS) simulations present significant challenges when applied to high-fidelity quantum circuits. This is primarily due to the difficulty in handling the high levels of entanglement found in such systems, as MPS is inherently limited by the bond dimension it can effectively manage. The high-fidelity and connectivity in quantum circuits, such as those involving H2, prove particularly formidable for MPS due to their ability to produce near-maximally entangled states. As a result, current MPS methods face limitations in scalability and fidelity, rendering them impractical for simulating complex quantum circuits at experimental fidelities.\n\nIn contrast, tensor-network (TN) methods offer a more general framework that can potentially approximate states with greater bipartite entanglement than MPS can handle. However, t